__ProPythia__

This file is a simulation fro the antioxidant dataset;

I intend to run propythia here so that in the future I can make comparisons with the results obtained with omnia.

In [ ]:
pip install propythia

i will do this simulation with the antioxidant data, where the data in unbalanced

# imports

In [ ]:
import sys
import os
import csv
import numpy as np
import pandas as pd

from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, matthews_corrcoef
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier

from propythia.protein.descriptors import ProteinDescritors
from propythia.protein.encoding import Encoding
from propythia.ml.shallow_ml import ShallowML
from propythia.protein.sequence import ReadSequence

from propythia.ml.deep_ml import DeepML

from propythia.feature_selection import FeatureSelection

from propythia.preprocess import Preprocess

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from itertools import product

# load the dataset

In [ ]:
antioxidant_data=pd.read_csv('data_antioxidant_test.csv')

In [ ]:
antioxidant_data["classe"].value_counts()

In [ ]:
# Get the maximum and minimum length of strings in the specified column
max_length = antioxidant_data["Seqs"].str.len().max()
min_length = antioxidant_data["Seqs"].str.len().min()
mean_seqs= antioxidant_data["Seqs"].str.len().mean()
num_sequences_over_600 = (antioxidant_data["Seqs"].str.len() > 600).sum()

print(f"Number of sequences over 600 amino acids: {num_sequences_over_600}")

num_sequences_under_100 = (antioxidant_data["Seqs"].str.len() < 100).sum()

print(f"Number of sequences under 100 amino acids: {num_sequences_under_100}")

print(f"Maximum length: {max_length}")
print(f"Minimum length: {min_length}")
print(f"mean: {mean_seqs}")


In [ ]:
mask = antioxidant_data["Seqs"].str.len() > 350

counts = antioxidant_data[mask].groupby("classe").size()

print(counts)

# Encoding

In [ ]:
#i think that is more complete, maybe need to change omnia or adapt omnia
def pad_and_truncate_sequences(df, seq_col, max_length, padding_value='X'):
    """
    Pad and truncate the protein sequences in a DataFrame to a specific length.

    :param df: DataFrame containing the protein sequences.
    :param seq_col: Name of the column in df that contains the protein sequences.
    :param max_length: The maximum length for all sequences.
    :param padding_value: The value to use for padding the sequences.
    :return: DataFrame with the padded and truncated sequences.
    """
    def pad_and_truncate(seq):
        # Truncate the sequence if it's too long
        if len(seq) > max_length:
            seq = seq[:max_length]
        # Pad the sequence if it's not long enough
        elif len(seq) < max_length:
            seq += padding_value * (max_length - len(seq))
        return seq

    df['padded_and_truncated_sequence'] = df[seq_col].apply(pad_and_truncate)
    return df

# Use the function
antioxidant_data = pad_and_truncate_sequences(antioxidant_data, 'Seqs', 350)

In [ ]:
enconde_df = Encoding(dataset= antioxidant_data ,  col= 'padded_and_truncated_sequence')
nlf=enconde_df.get_nlf()
print(np.array(nlf['nlf'][0]).shape)
print(nlf['nlf'])

expanded_arrays =  nlf['nlf'].apply(lambda x: np.array(x))
X_nlf = np.array(expanded_arrays.tolist())

y_nlf = nlf['classe']

__esm__

In [ ]:
enconde_df = Encoding(dataset= antioxidant_data ,  col= 'padded_and_truncated_sequence')

esm = enconde_df.get_esm(v_esm='esm2_150')

X_esm=esm ['esm']
y_esm = esm['classe']

__protbert__

In [ ]:
enconde_df = Encoding(dataset= antioxidant_data ,  col= 'padded_and_truncated_sequence')
protbert = enconde_df.get_protbert()

X_protbert = protbert['protbert']
y_protbert = protbert['classe']

# Obtain all the features 

In [ ]:
#calculate features
def calculate_feature(data):
    list_feature = []
    for seq in data['Seqs']:
        res = {'sequence': seq}
        sequence = ReadSequence()  # creating sequence object
        ps = sequence.read_protein_sequence(seq)
        protein = ProteinDescritors(ps)  # creating object to calculate descriptors
        feature = protein.get_adaptable([44])
        #feature_dict = feature.iloc[0].to_dict()
        res.update(feature)
        # feature = protein.get_all(lamda_paac=5, lamda_apaac=5) #minimal seq len = 5
        # lambda should not be larger than len(sequence)
        list_feature.append(res)
    print('saving features')
    antioxidant= pd.DataFrame(list_feature)
    return antioxidant

In [ ]:
antioxidant_data.head()

In [ ]:
antioxidant_dataset=calculate_feature(antioxidant_data)
antioxidant_dataset

In [ ]:
y_descriptors=antioxidant_data["classe"]
X_descriptors=antioxidant_dataset.loc[:,antioxidant_dataset.columns != "sequence"]

# split antioxidant_dataset X and Y 

# split in train and test

In [ ]:
X_train_descriptors, X_test_descriptors, y_train_descriptors, y_test_descriptors = train_test_split(X_descriptors,y_descriptors, test_size=0.2,stratify=y_descriptors, random_state=42)
X_train_nlf, X_test_nlf, y_train_nlf, y_test_nlf = train_test_split(X_nlf,y_nlf, test_size=0.2,stratify=y_nlf, random_state=42)
#X_train_esm, X_test_esm, y_train_esm, y_test_esm = train_test_split(X_esm,y_esm, test_size=0.2,stratify=y_esm, random_state=42)
#X_train_protbert, X_test_protbert, y_train_protbert, y_test_protbert = train_test_split(X_protbert,y_protbert, test_size=0.2,stratify=y_protbert, random_state=42)

# standard scaler article does not refer scaling and do not validate in x_test, however, we do it anyway
#scaler = StandardScaler().fit(X_train_descriptors)
#X_train_descriptors = scaler.transform(X_train_descriptors)
#X_test_descriptors = scaler.transform(X_test_descriptors)

In [ ]:
print('train_x', X_train_nlf.shape)
print('test_x', X_test_nlf.shape)

# ShallowML

In [ ]:
def train_multiple_models(X_train, X_test, y_train, y_test,X,model_names=['svm','linear_svm','rf','gboosting','knn','sgd','lr','nn']):
    """
    Train multiple models and return the results in a DataFrame.
    :param model_names: List with the names of the models to train.
    :param X_train: Training set features.
    :param X_test: Test set features.
    :param y_train: Training set labels.
    :param y_test: Test set labels.
    :param X: DataFrame with the features.
    :return: DataFrame with the results.
    """
    results = []
    ml = ShallowML(X_train, X_test, y_train, y_test, report_name=None, columns_names=X.columns)
    for model_name in model_names:
        if model_name == 'svm':
            model_params = {'clf__C':[0.01, 1.0,10], 'clf__kernel': 'rbf', 'clf__gamma': ['auto', 'scale'], 'clf__class_weight': ['None','balanced']}
        elif model_name == 'linear_svm':
            model_params={'clf__C':[0.01, 1.0,10], 'clf__penalty': 'l2', 'clf__class_weight': ['None','balanced']}
        elif model_name == 'rf':
            model_params = {'clf__n_estimators': [10,100,500], 'clf__max_features': [ 'sqrt', 'log2'],'clf__criterion':['gini','entropy'] , 'clf__class_weight': ['None','balanced']}
        elif model_name == 'gboosting':
            model_params = {'clf__n_estimators': [10, 100, 500], 'clf__max_depth': [1, 3, 5, 10],'clf__max_features': [0.6, 0.9],'clf__learning_rate':[0.1,1],}
        elif model_name == 'knn':
            model_params = {'clf__n_neighbors': [2,5,10,15],'clf__weights':['uniform', 'distance'], 'clf__leaf_size': [15, 30, 60]}
        elif model_name == 'sgd':
            model_params = {'clf__loss': ['hinge', 'log', 'modified_huber', 'perceptron'],'clf__alpha': [0.00001, 0.0001, 0.001, 0.01],'clf__early_stopping': [True],'clf__validation_fraction': [0.2],'clf__n_iter_no_change': [5,10],'clf__class_weight':['None','balanced'] }
        elif model_name== 'lr':
            model_params = {'clf__C': [0.01, 0.1, 1.0, 10.0], 'clf__solver': ['liblinear', 'lbfgs', 'sag'], 'clf__class_weight': ['None','balanced'],'clf__early_stopping': [True],'clf__validation_fraction': [0.2],'clf__n_iter_no_change': [5,10] }
        elif model_name== 'nn':
            model_params = {'clf__hidden_layer_sizes':[(50,),(100,),(200,)],'clf__activation': ['logistic', 'tanh', 'relu'],'clf__solver':['adam','sgd'],'clf__alpha': [0.00001, 0.0001, 0.001],'clf__learning_rate_init': [0.0001, 0.001, 0.01]}
        else:
            raise ValueError(f"Model {model_name} not implemented.")
        # Call the function train best model to train the model
        best_classifier = ml.train_best_model(model_name, score =make_scorer(matthews_corrcoef), param_grid=model_params,cv=5)
        scores, report, cm, cm2 = ml.score_testset(best_classifier)

        results.append({
            'model_name': model_name,
            'best_params': best_classifier,
            'scores': scores
        })
    
    results_df=pd.DataFrame(results)
        
    return results_df,best_classifier

#nao coloquei a dar as melhores features pois o propythia so tem isso implementado para alguns modelos e nao para todos



In [ ]:
results_df = train_multiple_models(X_train_descriptors, X_test_descriptors, y_train_descriptors, y_test_descriptors,antioxidant_data, model_names=['rf'])
results_df

# Deep Learning

O espaço de busca para a otimização de hiperparâmetros é fornecido pelo argumento param_grid. Se param_grid for None, a função irá buscar um conjunto padrão de hiperparâmetros, que é definido na função _get_opt_params.


In [ ]:
#cnn and propythia
def create_model(num_conv_layers=2, num_filters=64, filter_sizes=2, pool_size=2, num_dense_layers=2, neurons_dense=128, drop=0.3, activation='relu', last_layers_activations='sigmoid'):
    model = tf.keras.models.Sequential()

    # Convolutional layers
    for i in range(num_conv_layers):
        model.add(tf.keras.layers.Conv1D(filters=num_filters, kernel_size=filter_sizes, strides=1, padding='same', activation=activation))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling1D(pool_size, padding='same'))
        model.add(tf.keras.layers.Dropout(drop))

    # Flatten layer
    model.add(tf.keras.layers.Flatten())

    # Dense layers
    for i in range(num_dense_layers):
        model.add(tf.keras.layers.Dense(neurons_dense, activation=activation))
        model.add(tf.keras.layers.Dropout(drop))
    model.add(tf.keras.layers.Dense(1, activation=last_layers_activations))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

dl=DeepML(X_train_nlf, y_train_nlf, X_test_nlf, y_test_nlf, number_classes=2, problem_type='binary',
          x_dval=None, y_dval=None, epochs=100, batch_size=64,
          path='', report_name=None, verbose=1,
         early_stopping_patience=20, reduce_lr_patience=20, reduce_lr_factor=0.2, reduce_lr_min=0.00001,
                 )
model = KerasClassifier(build_fn=create_model, verbose=0)

def generate_param_grid(num_conv_layers, num_dense_layers):
    param_grid = {
        'num_conv_layers': [num_conv_layers],
        'num_filters': list(product([32, 64, 128], repeat=num_conv_layers)),
        'filter_sizes': [2, 3],
        'pool_size': [2, 3],
        'num_dense_layers': [num_dense_layers],
        'neurons_dense': list(product([32, 64, 128], repeat=num_dense_layers)),
        'drop': [0.1, 0.3,  0.5],
        'activation': ['relu'],
        'last_layers_activations': ['sigmoid']
    }

    return param_grid
    
num_conv_layers = 1
num_dense_layers = 2
param_grid = generate_param_grid(num_conv_layers, num_dense_layers)

dl.get_opt_params(param_grid,model,optType='gridSearch')

In [ ]:
#como ter tudo a correr , sem a necessidade de estar a mudar o numero de camdas convulacionas e dense
def generate_param_grids(num_conv_layers, num_dense_layers):
    param_grids = []
    for num_conv_layers in num_conv_layerse:
        for num_dense_layers in num_dense_layers:
            param_grid = {
                'num_conv_layers': [num_conv_layers],
                'num_filters': list(product([32, 64, 128], repeat=num_conv_layers)),
                'filter_sizes': [2, 3],
                'pool_size': [2, 3],
                'num_dense_layers': [num_dense_layers],
                'neurons_dense': list(product([32, 64, 128], repeat=num_dense_layers)),
                'drop': [0.1, 0.3,  0.5],
                'activation': ['relu'],
                'last_layers_activations': ['sigmoid']
            }
            param_grids.append(param_grid)
    return param_grids

conv_layers_range = range(1, 4)  
dense_layers_range = range(1, 4)  
param_grids = generate_param_grids(conv_layers_range, dense_layers_range)

for param_grid in param_grids:
    dl.get_opt_params(param_grid, model, optType='gridSearch')

# Dúvida : como funciona o los na omnia será que tb devo colocar como hiperparametro- acho que não
# devo fazer para o cnn-rnn hicrido tendo em conta o tipo de dados ( tabulares e não tabulares)

In [ ]:
#rnn and propythia
def create_rnn_model(rnn_type='LSTM', bidirectional=False, num_rnn_layers=3, hidden_dim=64, num_dense_layers=1, neurons_dense=32, output_dim=1, drop=0.3, activation='relu', last_layers_activations='sigmoid'):
    model = tf.keras.models.Sequential()

    # RNN layers
    for i in range(num_rnn_layers):
        if rnn_type == 'LSTM':
            rnn_layer = tf.keras.layers.LSTM(hidden_dim, return_sequences=(i != num_layers - 1), activation=activation)
        elif rnn_type == 'GRU':
            rnn_layer = tf.keras.layers.GRU(hidden_dim, return_sequences=(i != num_layers - 1), activation=activation)
        elif rnn_type == 'SimpleRNN':
            rnn_layer = tf.keras.layers.SimpleRNN(hidden_dim, return_sequences=(i != num_layers - 1), activation=activation)
        else:
            raise ValueError("Invalid RNN type. Supported types are 'LSTM', 'GRU', and 'SimpleRNN'.")

        if bidirectional:
            rnn_layer = tf.keras.layers.Bidirectional(rnn_layer)

        model.add(rnn_layer)
        model.add(tf.keras.layers.Dropout(drop))

    # Dense layers
    for i in range(num_dense_layers):
        model.add(tf.keras.layers.Dense(neurons_dense, activation=activation))
        model.add(tf.keras.layers.Dropout(drop))

    model.add(tf.keras.layers.Dense(output_dim, activation=last_layers_activations))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

dl=DeepML(X_train_nlf, y_train_nlf, X_test_nlf, y_test_nlf, number_classes=2, problem_type='binary',
          x_dval=None, y_dval=None, epochs=100, batch_size=64,
          path='', report_name=None, verbose=1,
         early_stopping_patience=20, reduce_lr_patience=20, reduce_lr_factor=0.2, reduce_lr_min=0.00001,
                 )
model = KerasClassifier(build_fn=create_model, verbose=0)

def generate_param_grid(num_rnn_layers, num_dense_layers):
    param_grid = {
        'rnn_type': ['LSTM', 'GRU', 'SimpleRNN'],
        'bidirectional': [True, False],
        'num_rnn_layers': [num_rnn_layers],
        'hidden_dim': list(product([32, 64, 128],repeat=num_rnn_layers)),
        'num_dense_layers': [num_dense_layers],
        'neurons_dense': list(product([32, 64, 128], repeat=num_dense_layers)),
        'output_dim': [1],
        'drop': [0.1, 0.3, 0.5],
        'activation': ['relu'],
        'last_layers_activations': ['sigmoid']
    }

    return param_grid
num_rnn_layers = 1
num_dense_layers = 2
param_grid = generate_param_grid(num_rnn_layers, num_dense_layers)

dl.get_opt_params(param_grid,model,optType='gridSearch')

In [ ]:
#autoencodermlp simultaneously and propythia
def create_autoencoder_mlp_model(latent_dim=12, num_layers=2, input_dim=100,neurons_per_layer=50, num_layers_class=2, neurons_mlp=10, num_classes=2, drop=0.3, activation="relu", last_layers_activations="sigmoid"):
    model = tf.keras.models.Sequential()

    # Encoder layers
    model.add(tf.keras.layers.Dense(input_dim, activation=activation))
    model.add(tf.keras.layers.Dropout(drop))
    for i in range(1, num_layers):
        model.add(tf.keras.layers.Dense(neurons_per_layer, activation=activation))
        model.add(tf.keras.layers.Dropout(drop))
    model.add(tf.keras.layers.Dense(latent_dim, activation=activation))
    model.add(tf.keras.layers.Dropout(drop))

    # Decoder layers
    for i in reversed(range(num_layers)):
        model.add(tf.keras.layers.Dense(neurons_per_layer, activation=activation))
        model.add(tf.keras.layers.Dropout(drop))
    model.add(tf.keras.layers.Dense(input_dim, activation=activation))

    # MLP layers
    for i in range(num_layers_class):
        model.add(tf.keras.layers.Dense(neurons_mlp, activation=activation))
        model.add(tf.keras.layers.Dropout(drop))
    model.add(tf.keras.layers.Dense(num_classes, activation=last_layers_activations))

    model.compile(loss=['mse', 'binary_crossentropy'],loss_weights=[1, 1], optimizer='adam', metrics=['accuracy'])
    return model

dl=DeepML(X_train_nlf, y_train_nlf, X_test_nlf, y_test_nlf, number_classes=2, problem_type='binary',
        x_dval=None, y_dval=None, epochs=100, batch_size=64,
        path='', report_name=None, verbose=1,
        early_stopping_patience=20, reduce_lr_patience=20, reduce_lr_factor=0.2, reduce_lr_min=0.00001,
                )
model = KerasClassifier(build_fn=create_model, verbose=0)

def generate_param_grid(num_layers, num_layers_class):
    param_grid = {
        'latent_dim': [12, 6],
        'num_layers': [num_layers],
        'input_dim': [100, 200, 300],#devia ser o n_features
        'neurons_per_layer': list(product([25, 50, 125]),repeat=num_layers),
        'num_layers_class': [num_layers_class],
        'neurons_mlp': list(product([10,15,20]),repeat=num_layers_class),
        'num_classes': [2],
        'drop': [0.1, 0.2, 0.3],
        'activation': ['relu'],
        'last_layers_activations': ['sigmoid']
    }
    return param_grid

num_layers = 2
num_layers_class = 2
param_grid = generate_param_grid(num_layers, num_layers_class)

dl.get_opt_params(param_grid,model,optType='gridSearch')


Para o hibrido nao vale a pena!! Alias acho que o hibrido nem faz muito sentido no tipo de dados que temos logo tb devemos ter isso em consideração na omnia